In [18]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Libabry**

In [19]:
## Imports
import os
import sys
import random
import numpy as np
from numpy import save
from numpy import load
#
import pandas as pd
import seaborn as sns
from PIL import Image
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
#
import cv2
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, 
    BatchNormalization,
    Activation,
    Dense,
    Dropout,
    MaxPooling2D, 
    GlobalMaxPool2D,
    Conv2D,
    Conv2DTranspose,
    UpSampling2D,
    concatenate,
    add,
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

## Seeding
seed=2019
random.seed=seed
np.random.seed=seed
tf.seed=seed

# **Unet 5_101**

In [20]:
# I tried to make it as simple as possible, I think you can understand the architecture just by looking at the code below
def UNet5101(input_img):

    c1 = Conv2D(16, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(input_img)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    
    c1 = Conv2D(16, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(0.1)(p1)
#-----------------------------------------------------------------------------------------------------------------    
    
    c2 = Conv2D(32, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    
    c2 = Conv2D(32, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(0.1)(p2) 
#-----------------------------------------------------------------------------------------------------------------    
    
    c3 = Conv2D(64, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    
    c3 = Conv2D(64, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(0.1)(p3) 
    
#-----------------------------------------------------------------------------------------------------------------    
    
    c4 = Conv2D(128, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    
    c4 = Conv2D(128, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(0.1)(p4) 
    
#-----------------------------------------------------------------------------------------------------------------    
    
    c5 = Conv2D(256, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    
    c5 = Conv2D(256, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    
#------UMSAMPLE STARTS---------------------------------------------------------------------------------------------

    u6 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(0.1)(u6)
    
    c6 = Conv2D(128, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)
    
    c6 = Conv2D(128, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)
    
#---------------------------------------------------------------------------------------------------

    u7 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'valid')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(0.1)(u7)
    
    c7 = Conv2D(64, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)
    
    c7 = Conv2D(64, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)
    
#---------------------------------------------------------------------------------------------------

    u8 = Conv2DTranspose(32, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(0.1)(u8)
    
    c8 = Conv2D(32, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)

    c8 = Conv2D(32, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)
    
#---------------------------------------------------------------------------------------------------

    u9 = Conv2DTranspose(16, (3, 3), strides = (2, 2), padding = 'valid')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(0.1)(u9)
    
    c9 = Conv2D(16, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)
    
    c9 = Conv2D(16, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(c9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)
  
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)    
    model = Model(input_img, outputs)
    return model

In [21]:
input_img=Input((101, 101, 1), name='img')
model=UNet5101(input_img)
#model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=[iou_metric])

# **Unet 6_128**

In [22]:
# I tried to make it as simple as possible, I think you can understand the architecture just by looking at the code below
def UNet6128(input_img):
    
    #inputs = Input(input_size)

    c1 = Conv2D(32, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(input_img)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    
    c1 = Conv2D(32, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    
    p1 = MaxPooling2D(pool_size=(2, 2))(c1)
    #p1 = Dropout(0.1)(p1)
#-----------------------------------------------------------------------------------------------------------------    
    
    c2 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    
    c2 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    
    p2 = MaxPooling2D(pool_size=(2, 2))(c2)
    #p2 = Dropout(0.1)(p2) 
#-----------------------------------------------------------------------------------------------------------------    
    
    c3 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    
    c3 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    
    p3 = MaxPooling2D(pool_size=(2, 2))(c3)
    #p3 = Dropout(0.1)(p3) 
    
#-----------------------------------------------------------------------------------------------------------------    
    
    c4 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    
    c4 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    #p4 = Dropout(0.1)(p4) 
    
#-----------------------------------------------------------------------------------------------------------------    
    
    c5 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    
    c5 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)

    p5 = MaxPooling2D(pool_size=(2, 2))(c5)
    #p5 = Dropout(0.1)(p5)


#-------------------------------------------------------BRIDGE----------------------------------------------------------    
    
    c6 = Conv2D(1024, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p5)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)
    
    c6 = Conv2D(1024, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)

    #p6 = MaxPooling2D((2, 2))(c6)
    #p6 = Dropout(0.1)(p6)


    
#------UPSAMPLE STARTS---------------------------------------------------------------------------------------------
    u1 = UpSampling2D(size=(2,2))(c6)
    u1 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u1)
    u1 = BatchNormalization()(u1)
    u1 = Activation('relu')(u1)


    merge_u1 = concatenate([u1, c5])
    #u1 = Dropout(0.1)(u6)
    
    cu1 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u1)
    cu1 = BatchNormalization()(cu1)
    cu1 = Activation('relu')(cu1)
    
    cu1 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu1)
    cu1 = BatchNormalization()(cu1)
    cu1 = Activation('relu')(cu1)
    
#---------------------------------------------------------------------------------------------------

    u2 = UpSampling2D(size=(2,2))(cu1)
    u2 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u2)
    u2 = BatchNormalization()(u2)
    u2 = Activation('relu')(u2)


    merge_u2 = concatenate([u2, c4])
    #u1 = Dropout(0.1)(u6)
    
    cu2 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u2)
    cu2 = BatchNormalization()(cu2)
    cu2 = Activation('relu')(cu2)
    
    cu2 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu2)
    cu2 = BatchNormalization()(cu2)
    cu2 = Activation('relu')(cu2)
    
#---------------------------------------------------------------------------------------------------

    u3 = UpSampling2D(size=(2,2))(cu2)
    u3 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u3)
    u3 = BatchNormalization()(u3)
    u3 = Activation('relu')(u3)


    merge_u3 = concatenate([u3, c3])
    #u1 = Dropout(0.1)(u6)
    
    cu3 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u3)
    cu3 = BatchNormalization()(cu3)
    cu3 = Activation('relu')(cu3)
    
    cu3 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu3)
    cu3 = BatchNormalization()(cu3)
    cu3 = Activation('relu')(cu3)
    
#---------------------------------------------------------------------------------------------------

    u4 = UpSampling2D(size=(2,2))(cu3)
    u4 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u4)
    u4 = BatchNormalization()(u4)
    u4 = Activation('relu')(u4)


    merge_u4 = concatenate([u4, c2])
    #u1 = Dropout(0.1)(u6)
    
    cu4 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u4)
    cu4 = BatchNormalization()(cu4)
    cu4 = Activation('relu')(cu4)
    
    cu4 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu4)
    cu4 = BatchNormalization()(cu4)
    cu4 = Activation('relu')(cu4)
    
#---------------------------------------------------------------------------------------------------

    u5 = UpSampling2D(size=(2,2))(cu4)
    u5 = Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u5)
    u5 = BatchNormalization()(u5)
    u5 = Activation('relu')(u5)


    merge_u5 = concatenate([u5, c1])
    #u1 = Dropout(0.1)(u6)
    
    cu5 = Conv2D(32, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u5)
    cu5 = BatchNormalization()(cu5)
    cu5 = Activation('relu')(cu5)
    
    cu5 = Conv2D(32, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu5)
    cu5 = BatchNormalization()(cu5)
    cu5 = Activation('relu')(cu5)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(cu5)

    model = Model(input_img, outputs)
    return model

In [23]:
input_img=Input((128, 128, 1), name='img')
model=UNet6128(input_img)
#model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=[iou_metric])

# **Unet Vgg16**

In [24]:
# I tried to make it as simple as possible, I think you can understand the architecture just by looking at the code below
def UNet112(input_img):
    
    #inputs = Input(input_size)

    c1 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(input_img)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    
    c1 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    
    p1 = MaxPooling2D(pool_size=(2, 2))(c1)
    #p1 = Dropout(0.1)(p1)
#-----------------------------------------------------------------------------------------------------------------    
    
    c2 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    
    c2 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    
    p2 = MaxPooling2D(pool_size=(2, 2))(c2)
    #p2 = Dropout(0.1)(p2) 
#-----------------------------------------------------------------------------------------------------------------    
    
    c3 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    
    c3 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)

    c3 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    
    p3 = MaxPooling2D(pool_size=(2, 2))(c3)
    #p3 = Dropout(0.1)(p3) 
    
#-----------------------------------------------------------------------------------------------------------------    
    
    c4 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    
    c4 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)

    c4 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    #p4 = Dropout(0.1)(p4) 
    
#-------------------------------------------------------BRIDGE----------------------------------------------------------    
    
    c5 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    
    c5 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)

    c5 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)

    #p5 = MaxPooling2D(pool_size=(2, 2))(c5)
    #p5 = Dropout(0.1)(p5)


#------UPSAMPLE STARTS-----------------------------------------------------------------------------------------------------------    
    
    u1 = UpSampling2D(size=(2,2))(c5)
    u1 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u1)
    u1 = BatchNormalization()(u1)
    u1 = Activation('relu')(u1)
    
    merge_u1 = concatenate([u1, c4])

    cu1 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u1)
    cu1 = BatchNormalization()(cu1)
    cu1 = Activation('relu')(cu1)
    
    cu1 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu1)
    cu1 = BatchNormalization()(cu1)
    cu1 = Activation('relu')(cu1)

    cu1 = Conv2D(512, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu1)
    cu1 = BatchNormalization()(cu1)
    cu1 = Activation('relu')(cu1)




    
#--------------------------------------------------------------------------------------------------------------------------
    u2 = UpSampling2D(size=(2,2))(cu1)
    u2 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u2)
    u2 = BatchNormalization()(u2)
    u2 = Activation('relu')(u2)


    merge_u2 = concatenate([u2, c3])
    #u1 = Dropout(0.1)(u6)
    
    cu2 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u2)
    cu2 = BatchNormalization()(cu2)
    cu2 = Activation('relu')(cu2)
    
    cu2 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu2)
    cu2 = BatchNormalization()(cu2)
    cu2 = Activation('relu')(cu2)

    cu2 = Conv2D(256, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu2)
    cu2 = BatchNormalization()(cu2)
    cu2 = Activation('relu')(cu2)
    
    
#---------------------------------------------------------------------------------------------------

    u3 = UpSampling2D(size=(2,2))(cu2)
    u3 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u3)
    u3 = BatchNormalization()(u3)
    u3 = Activation('relu')(u3)


    merge_u3 = concatenate([u3, c2])
    #u1 = Dropout(0.1)(u6)
    
    cu3 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u3)
    cu3 = BatchNormalization()(cu3)
    cu3 = Activation('relu')(cu3)
    
    cu3 = Conv2D(128, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu3)
    cu3 = BatchNormalization()(cu3)
    cu3 = Activation('relu')(cu3)
    
#---------------------------------------------------------------------------------------------------

    u4 = UpSampling2D(size=(2,2))(cu3)
    u4 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(u4)
    u4 = BatchNormalization()(u4)
    u4 = Activation('relu')(u4)


    merge_u4 = concatenate([u4, c1])
    #u1 = Dropout(0.1)(u6)
    
    cu4 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(merge_u4)
    cu4 = BatchNormalization()(cu4)
    cu4 = Activation('relu')(cu4)
    
    cu4 = Conv2D(64, kernel_size = (3,3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')(cu4)
    cu4 = BatchNormalization()(cu4)
    cu4 = Activation('relu')(cu4)

    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(cu4)

    model = Model(input_img, outputs)
    return model

In [25]:
input_img=Input((112, 112, 1), name='img')
model=UNet112(input_img)
#model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=[iou_metric])

# **ResUnet**

In [26]:
def res_block(x,filter_size,size):
  conv = Conv2D(size,kernel_size=(filter_size,filter_size),padding='same')(x)
  conv = BatchNormalization()(conv)
  conv = Activation('relu')(conv)

  conv = Conv2D(size,kernel_size=(filter_size,filter_size),padding='same')(conv)
  conv = BatchNormalization()(conv)
  
  shortcut = Conv2D(size,kernel_size=(1,1),padding='same')(x)
  shortcut = BatchNormalization()(shortcut)

  res = add([shortcut,conv])
  res = Activation('relu')(res)

  return res

In [27]:
# I tried to make it as simple as possible, I think you can understand the architecture just by looking at the code below
def Res_UNet(input_img):

    c1 = Conv2D(16, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(input_img)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    
    c1 = res_block(c1,3,16)
    #c1 = BatchNormalization()(c1)
    #c1 = Activation('relu')(c1)
    
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(0.1)(p1)
#-----------------------------------------------------------------------------------------------------------------    
    
    c2 = Conv2D(32, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    
    c2 = res_block(c2,3,32)
    #c2 = BatchNormalization()(c2)
    #c2 = Activation('relu')(c2)
    
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(0.1)(p2) 
#-----------------------------------------------------------------------------------------------------------------    
    
    c3 = Conv2D(64, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    
    c3 = res_block(c3,3,64)
    #c3 = BatchNormalization()(c3)
    #c3 = Activation('relu')(c3)
    
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(0.1)(p3) 
    
#-----------------------------------------------------------------------------------------------------------------    
    
    c4 = Conv2D(128, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    
    c4 = res_block(c4,3,128)
    #c4 = BatchNormalization()(c4)
    #c4 = Activation('relu')(c4)
    
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(0.1)(p4) 
    
#-----------------------------------------------------------------------------------------------------------------    
    
    c5 = Conv2D(256, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    
    c5 = res_block(c5,3,256)
    #c5 = BatchNormalization()(c5)
    #c5 = Activation('relu')(c5)
    
#------UMSAMPLE STARTS---------------------------------------------------------------------------------------------

    u6 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(0.1)(u6)
    
    c6 = Conv2D(128, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)
    
    c6 = res_block(c6,3,128)
    #c6 = BatchNormalization()(c6)
    #c6 = Activation('relu')(c6)
    
#---------------------------------------------------------------------------------------------------

    u7 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'valid')(c6)
    #u7 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(0.1)(u7)
    
    c7 = Conv2D(64, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)
    
    c7 = res_block(c7,3,64)
    #c7 = BatchNormalization()(c7)
    #c7 = Activation('relu')(c7)
    
#---------------------------------------------------------------------------------------------------

    u8 = Conv2DTranspose(32, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(0.1)(u8)
    
    c8 = Conv2D(32, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)

    c8 = res_block(c8,3,32)
    #c8 = BatchNormalization()(c8)
    #c8 = Activation('relu')(c8)
    
#---------------------------------------------------------------------------------------------------

    u9 = Conv2DTranspose(16, (3, 3), strides = (2, 2), padding = 'valid')(c8)
    #u9 = Conv2DTranspose(16, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(0.1)(u9)
    
    c9 = Conv2D(16, kernel_size = (3,3), kernel_initializer = 'he_normal', padding = 'same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)
    
    c9 = res_block(c9,3,16)
    #c9 = BatchNormalization()(c9)
    #c9 = Activation('relu')(c9)
    
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

 
    
    model = Model(input_img, outputs)
    return model

In [28]:
input_img=Input((101, 101, 1), name='img')
model=Res_UNet(input_img)
#model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=[iou_metric])

# **File submission**

In [13]:
from tqdm import tqdm_notebook, tnrange
from skimage.transform import resize

In [14]:
im_width = 101
im_height = 101
im_chan = 1
path_test = '/content/drive/MyDrive/ImageProcessing/test/'
test_ids = next(os.walk(path_test+"images"))[2]

In [15]:
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan))
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    sizes_test.append([x.shape[0], x.shape[1]])
    x = resize(x, (im_width, im_height, 1), mode='constant', preserve_range=True)/255
    X_test[n] = x

Getting and resizing test images ... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/18000 [00:00<?, ?it/s]

**Load Weight For Model to Submit**

In [30]:
model.load_weights('/content/drive/MyDrive/Code For Report CS406.M11/GUI/Res_Unet_101_GD.h5')

**Run on Test Folder**

In [31]:
preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

563/563 [==============================] - 47s 67ms/step


In [32]:
preds_test_upsampled = []
for i in tnrange(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test_t[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), mode='constant', preserve_range=True))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


  0%|          | 0/18000 [00:00<?, ?it/s]

In [33]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1
    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0
    if format:
        z = ''
        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs
pred_dict = {fn[:-4]:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_notebook(enumerate(test_ids))}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

In [34]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')